# NERDS Data Analysis with Python and Pandas

In [2]:
import pandas as pd
import numpy as np
#these are the libraries needed to do this work. Are there more?

This project is intended to gauge whether there are funding discrepancies within Bexar County Independent School Districts when it comes to per-pupil spending and economic disadvantage rates. <br> 
If a discrepancy is detected (low correlation coefficient…threshold TBD…or there are outlier schools on the resulting scatterplot, we can work with an education reporter to investigate further. <br>
The data come from two sources: 
- The NERD$ data come from Georgetown University's [National Education Resource Database on School Spending Organization](https://georgetown.app.box.com/s/1dknmu4bjltrehzdygh63xnzebcki4ni/file/1060770984817) and represent the 2018-2019 academic year. <br>
- The Economic Disadvantage Data comes from [Square Meals.org](https://data.texas.gov/dataset/School-Nutrition-Programs-Contact-Information-and-/jezb-2499) and also represent the 2018-2019 academic year.<br>

# Data Import

In [3]:
nerds_full = pd.read_csv("essential-data/tx-nerds-2018_19.csv", dtype={'schoolid_stateassigned':str})
ed_full = pd.read_csv("essential-data/tx-sch_lunch-2018-19.csv", dtype={'SiteID':str, 'distid':str})

ed_full['school_id'] = ed_full['distid'] + ed_full['SiteID'].str.zfill(3)

In [ ]:
ed_full.groupby(['SiteCounty','distid'])['SiteISP'].size()

In [8]:
#I don't know how to quickly make this: I performed the groupby in the previous cell, cut and pasted it into these brackets, and then deleted all the other crap. It seems like there should have been another way.
#Allie cut and pasted from Excel. 
isds_tokeep = [130901, 15901, 15904, 15907, 15908, 15909, 15910, 15911, 15912, 15913, 15914, 15915, 15916, 15917, 163908, 46902, 46902, 94902, 130901, 163908]

In [7]:
ed_bexar = ed_full.loc[ed_full['distid'].isin(isds_tokeep)]

In [10]:
ed_bexar.columns

Index(['CEID', 'distname', 'SiteID', 'schoolname', 'distid', 'CECounty',
       'SiteCounty', 'SiteISP', 'school_id'],
      dtype='object')

In [11]:
nerds_full.columns

Index(['distid', 'schoolid_stateassigned', 'distname', 'schoolname',
       'enroll_raw_TX', 'pp_stloc_raw_TX'],
      dtype='object')

In [12]:
ed_full.columns

Index(['CEID', 'distname', 'SiteID', 'schoolname', 'distid', 'CECounty',
       'SiteCounty', 'SiteISP', 'school_id'],
      dtype='object')

In [13]:
complete_data = pd.merge(nerds_full, ed_full, left_on='schoolid_stateassigned',right_on='school_id',how='right',suffixes=('_nerds','_ed'))

In [14]:
complete_data.to_csv("/Users/wratko/Documents/projects/tx-school_funding/jupyter-notebooks/essential-data/complete_data.csv")

In [19]:
complete_data.columns

Index(['distid_nerds', 'schoolid_stateassigned', 'distname_nerds',
       'schoolname_nerds', 'enroll_raw_TX', 'pp_stloc_raw_TX', 'CEID',
       'distname_ed', 'SiteID', 'schoolname_ed', 'distid_ed', 'CECounty',
       'SiteCounty', 'SiteISP', 'school_id'],
      dtype='object')

In [20]:
list(complete_data.loc[~(pd.to_numeric(complete_data['pp_stloc_raw_TX'],errors='coerce').notnull())]['schoolname_ed'])

['HISD BOOT DAEP MS',
 'TEJEDA JUNIOR ACADEMY',
 'HISD BOOT DAEP HS',
 'HAC DAEP MS',
 'COLLIER DAEP',
 'LUCKEY RANCH ELEMENTARY',
 'BEXAR COUNTY JJAEP',
 'LOSOYA MIDDLE SCHOOL']

In [24]:
complete_data = complete_data.loc[pd.to_numeric(complete_data['pp_stloc_raw_TX'], errors='coerce').notnull()]

In [25]:
complete_data['pp_stloc_raw_TX'] = complete_data['pp_stloc_raw_TX'].astype(float)

In [29]:
bexar_corr = complete_data.groupby('distname_nerds')['pp_stloc_raw_TX','SiteISP'].corr().reset_index()
bexar_corr = bexar_corr.loc[bexar_corr['level_1'] == 'pp_stloc_raw_TX']
bexar_corr = bexar_corr[['distname_nerds','level_1','SiteISP']]

/var/folders/wg/0c8vsdgx6cj6d68fjsm44xnh0000gq/T/ipykernel_18479/2155754127.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bexar_corr = complete_data.groupby('distname_nerds')['pp_stloc_raw_TX','SiteISP'].corr().reset_index()


In [30]:
bexar_corr.sort_values('SiteISP', ascending=False)

,distname_nerds,level_1,SiteISP
6,EAST CENTRAL ISD,pp_stloc_raw_TX,0.717635
30,SOUTHSIDE ISD,pp_stloc_raw_TX,0.658392
0,ALAMO HEIGHTS ISD,pp_stloc_raw_TX,0.553086
24,SCHERTZ-CIBOLO-U CITY ISD,pp_stloc_raw_TX,0.456589
18,NORTH EAST ISD,pp_stloc_raw_TX,0.352962
12,JUDSON ISD,pp_stloc_raw_TX,0.281779
32,SOUTHWEST ISD,pp_stloc_raw_TX,0.271387
28,SOUTH SAN ANTONIO ISD,pp_stloc_raw_TX,0.220388
16,MEDINA VALLEY ISD,pp_stloc_raw_TX,0.166267
4,COMAL ISD,pp_stloc_raw_TX,0.093254
